In [9]:
import requests
import pandas as pd
from datetime import datetime
import logging
from typing import List, Dict, Any

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class SimpleInterparkCrawler:
    """심플 인터파크 HOT 티켓 크롤러"""
    
    def __init__(self):
        self.base_url = "https://tickets.interpark.com/contents/api/open-notice/notice-list"
        logger.info("인터파크 크롤러 초기화 완료")
    
    def _get_request_headers(self) -> Dict[str, str]:
        """API 요청 헤더"""
        return {
            "host": "tickets.interpark.com",
            "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36",
            "accept": "application/json, text/plain, */*",
            "referer": "https://tickets.interpark.com/contents/notice",
            "accept-language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7"
        }
    
    def _get_request_params(self) -> Dict[str, Any]:
        """API 요청 파라미터"""
        return {
            "goodsGenre": "ALL", 
            "goodsRegion": "ALL",
            "offset": 0,
            "pageSize": 1000,
            "sorting": "OPEN_ASC"
        }
    
    def fetch_ticket_data(self) -> List[Dict[str, Any]]:
        """인터파크 API에서 티켓 데이터 가져오기"""
        try:
            response = requests.get(
                self.base_url, 
                params=self._get_request_params(), 
                headers=self._get_request_headers(),
                timeout=30
            )
            response.raise_for_status()
            
            data = response.json()
            logger.info(f"API 호출 성공: {len(data)}개 티켓 조회")
            return data
            
        except requests.RequestException as e:
            logger.error(f"API 호출 실패: {e}")
            raise
    
    def filter_hot_tickets(self, raw_data: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """HOT 티켓만 필터링하고 정보 정리"""
        hot_tickets = []
        
        for ticket in raw_data:
            # HOT 티켓만 선별
            if ticket.get('goodsGenreStr', '') == '뮤지컬':
                if not ticket.get('isHot', False):
                    continue
            else:
                if not ticket.get('viewCount', 0) > 1000:
                    continue

            
            # 티켓 정보 정리
            ticket_info = {
                '오픈시간': ticket.get('openDateStr', ''),
                '조회수': ticket.get('viewCount', 0),
                '예매타입': ticket.get('openTypeStr', ''),
                '제목': ticket.get('title', ''),
                '예매코드': ticket.get('goodsCode', ''),
                '멀티오픈': ticket.get('hasMultipleOpenDates', False),
                '장르': ticket.get('goodsGenreStr', ''),
                '지역': ticket.get('goodsRegionStr', ''),
                '공연장': ticket.get('venueName', ''),
                '포스터URL': ticket.get('posterImageUrl', ''),
                '크롤링시간': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }
            hot_tickets.append(ticket_info)
        
        logger.info(f"HOT 티켓 {len(hot_tickets)}개 필터링 완료")
        return hot_tickets
    
    def save_to_csv(self, tickets: List[Dict[str, Any]], filename: str = None) -> str:
        """티켓 데이터를 CSV 파일로 저장"""
        if not filename:
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            filename = f"interpark_hot_tickets_{timestamp}.csv"
        
        try:
            df = pd.DataFrame(tickets)
            df.to_csv(filename, index=False, encoding='utf-8-sig')
            logger.info(f"CSV 파일 저장 완료: {filename}")
            return filename
        except Exception as e:
            logger.error(f"CSV 저장 실패: {e}")
            return ""
    
    def crawl(self) -> Dict[str, Any]:
        """크롤링 실행"""
        logger.info("인터파크 HOT 티켓 크롤링 시작")
        
        try:
            # 1. 티켓 데이터 가져오기
            raw_data = self.fetch_ticket_data()
            
            # 2. HOT 티켓 필터링
            hot_tickets = self.filter_hot_tickets(raw_data)
            
            # 3. CSV 저장
            csv_filename = ""
            if hot_tickets:
                csv_filename = self.save_to_csv(hot_tickets)
            
            # 4. 결과 반환
            result = {
                '전체티켓수': len(raw_data),
                'HOT티켓수': len(hot_tickets),
                'CSV파일': csv_filename,
                '크롤링시간': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                '티켓데이터': hot_tickets
            }
            
            logger.info(f"크롤링 완료: 전체 {len(raw_data)}개 중 HOT {len(hot_tickets)}개")
            return result
            
        except Exception as e:
            logger.error(f"크롤링 실행 오류: {e}")
            raise
    
    def print_summary(self, result: Dict[str, Any]):
        """크롤링 결과 요약 출력"""
        print("=" * 60)
        print("🎫 인터파크 HOT 티켓 크롤링 결과")
        print("=" * 60)
        print(f"📊 전체 티켓: {result['전체티켓수']}개")
        print(f"🔥 HOT 티켓: {result['HOT티켓수']}개")
        print(f"📁 CSV 파일: {result['CSV파일']}")
        print(f"⏰ 크롤링 시간: {result['크롤링시간']}")
        print("=" * 60)
        
        # HOT 티켓 목록 출력
        if result['티켓데이터']:
            print("\n🔥 HOT 티켓 목록:")
            for i, ticket in enumerate(result['티켓데이터'], 1):
                print(f"{i:2d}. {ticket['제목']}")
                print(f"    ⏰ {ticket['오픈시간']} | 👀 {ticket['조회수']:,}회 | 🎭 {ticket['장르']}")
                print(f"    🎪 {ticket['공연장']} | 🎫 {ticket['예매코드']}")
                print()

"""메인 실행 함수"""
try:
    print("🎫 인터파크 HOT 티켓 크롤러 시작\n")
    
    # 크롤러 생성
    crawler = SimpleInterparkCrawler()
    
    # 크롤링 실행
    result = crawler.crawl()
    
    # 결과 출력
    crawler.print_summary(result)
    
        
except KeyboardInterrupt:
    print("\n\n⏹️ 사용자에 의해 중단되었습니다.")
except Exception as e:
    print(f"\n❌ 오류 발생: {e}")
    logger.error(f"메인 실행 오류: {e}")


2025-07-12 10:48:26,467 - INFO - 인터파크 크롤러 초기화 완료
2025-07-12 10:48:26,468 - INFO - 인터파크 HOT 티켓 크롤링 시작
2025-07-12 10:48:26,521 - INFO - API 호출 성공: 68개 티켓 조회
2025-07-12 10:48:26,522 - INFO - HOT 티켓 9개 필터링 완료
2025-07-12 10:48:26,524 - INFO - CSV 파일 저장 완료: interpark_hot_tickets_20250712_104826.csv
2025-07-12 10:48:26,524 - INFO - 크롤링 완료: 전체 68개 중 HOT 9개


🎫 인터파크 HOT 티켓 크롤러 시작

🎫 인터파크 HOT 티켓 크롤링 결과
📊 전체 티켓: 68개
🔥 HOT 티켓: 9개
📁 CSV 파일: interpark_hot_tickets_20250712_104826.csv
⏰ 크롤링 시간: 2025-07-12 10:48:26

🔥 HOT 티켓 목록:
 1. 2025 영탁 단독 콘서트 "TAK SHOW4" - 전주
    ⏰ 2025-07-14 20:00:00 | 👀 1,850회 | 🎭 콘서트
    🎪 한국소리문화의전당 야외공연장 | 🎫 

 2. 2025 AKMU STANDING CONCERT ［악동들］ 
    ⏰ 2025-07-15 20:00:00 | 👀 34,896회 | 🎭 콘서트
    🎪 명화라이브홀 | 🎫 

 3. 뮤 내한공연 (MEW The Farewell Shows) 
    ⏰ 2025-07-15 20:00:00 | 👀 1,600회 | 🎭 콘서트
    🎪 노들섬 라이브하우스 | 🎫 25009793

 4. 그랜드 민트 페스티벌 2025 
    ⏰ 2025-07-16 18:00:00 | 👀 2,920회 | 🎭 콘서트
    🎪 올림픽공원 | 🎫 

 5. 너드커넥션 SUMMER LIVE, PULSE
    ⏰ 2025-07-17 18:00:00 | 👀 3,921회 | 🎭 콘서트
    🎪 무신사개러지 | 🎫 25008903

 6. 더 로즈(The Rose) Once Upon A WRLD Tour in Seoul 
    ⏰ 2025-07-23 12:00:00 | 👀 2,740회 | 🎭 콘서트
    🎪 블루스퀘어 SOL트래블홀 | 🎫 

 7. Invitation from Gensokyo 2025 ~ Midnight Concerto (동방프로젝트 오케스트라 콘서트) 
    ⏰ 2025-07-23 19:00:00 | 👀 2,793회 | 🎭 클래식/오페라
    🎪 KBS홀 | 🎫 

 8. 민트페스타 vol.78 SPIRITED
    ⏰ 2025-07-25 18:00:00 | 👀 4,027회

In [ ]:
result['티켓데이터'][0]

NameError: name 'hottickets' is not defined

In [8]:
import os
import requests
import json
import time
from datetime import datetime, timedelta
from dotenv import load_dotenv

url = "https://tickets.interpark.com/contents/api/open-notice/notice-list"

params = {
    "goodsGenre": "ALL",
    "goodsRegion": "ALL",
    "offset": 0,
    "pageSize": 50,
    "sorting": "OPEN_ASC"
}

headers = {
    "host": "tickets.interpark.com",
    "sec-ch-ua-platform": "Windows",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36",
    "accept": "application/json, text/plain, */*",
    "sec-ch-ua": "\"Chromium\";v=\"134\", \"Not:A-Brand\";v=\"24\", \"Google Chrome\";v=\"134\"",
    "sec-ch-ua-mobile": "?0",
    "sec-fetch-site": "same-origin",
    "sec-fetch-mode": "cors",
    "sec-fetch-dest": "empty",
    "referer": "https://tickets.interpark.com/contents/notice",
    "accept-encoding": "gzip, deflate, br, zstd",
    "accept-language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7"
}

try:
    response = requests.get(url, params=params, headers=headers)
    data = response.json()
    today_date = datetime.now().strftime('%Y년 %m월 %d일')
    tomorrow = (datetime.now() + timedelta(days=1)).date()
    
    print(f"=== {today_date} 티켓 오픈 정보 ===")
    message = f"<b>🎫 {today_date} 티켓 오픈 정보 🎫</b>\n\n"
    
    for ticket in data:
        open_time = ticket['openDateStr'][11:16]
        title = ticket['title']
        if len(title) > 40:
            title = title[:40] + "..."
        view_count = ticket['viewCount']
        goods_code = ticket['goodsCode']
        open_type = ticket['openTypeStr']
        
        # 날짜 문자열에서 연도, 월, 일 추출
        open_date_str = ticket['openDateStr'][:10]
        year, month, day = map(int, open_date_str.split('-'))
        ticket_date = datetime(year, month, day).date()
        
        # 날짜별 구분 및 서식 추가
        today = datetime.now().date()
        # 내일 날짜인지 확인 (오늘과 내일 티켓만 표시)
        if ticket_date > today + timedelta(days=1):
            break
        if ticket_date == today:
            date_emoji = "🔴 오늘"
        elif ticket_date == today + timedelta(days=1):
            date_emoji = "🟠 내일"
        else:
            date_emoji = f"⚪ {month}월 {day}일"
        
        # 각 티켓 정보를 깔끔하게 포맷팅
        message += f"<b>{date_emoji} [{open_time}]</b>\n"
        message += f"<b>{title}</b>\n"
        message += f"👁 조회수: {view_count}  |  🎟 예매코드: <code>{goods_code}</code>  |  📌{open_type}\n"
        message += "───────────────────\n"
        print(message)
except Exception as e:
    print(f"오류 발생: {e}")








=== 2025년 06월 15일 티켓 오픈 정보 ===
<b>🎫 2025년 06월 15일 티켓 오픈 정보 🎫</b>

<b>🟠 내일 [11:00]</b>
<b>뮤지컬 〈사랑의 하츄핑〉 - 서울앵콜 </b>
👁 조회수: 1109  |  🎟 예매코드: <code>25008505</code>  |  📌일반예매
───────────────────

<b>🎫 2025년 06월 15일 티켓 오픈 정보 🎫</b>

<b>🟠 내일 [11:00]</b>
<b>뮤지컬 〈사랑의 하츄핑〉 - 서울앵콜 </b>
👁 조회수: 1109  |  🎟 예매코드: <code>25008505</code>  |  📌일반예매
───────────────────
<b>🟠 내일 [13:00]</b>
<b>뮤지컬 〈사랑은 비를 타고〉 30주년 공연 </b>
👁 조회수: 1572  |  🎟 예매코드: <code>25004150</code>  |  📌마지막 티켓오픈
───────────────────

<b>🎫 2025년 06월 15일 티켓 오픈 정보 🎫</b>

<b>🟠 내일 [11:00]</b>
<b>뮤지컬 〈사랑의 하츄핑〉 - 서울앵콜 </b>
👁 조회수: 1109  |  🎟 예매코드: <code>25008505</code>  |  📌일반예매
───────────────────
<b>🟠 내일 [13:00]</b>
<b>뮤지컬 〈사랑은 비를 타고〉 30주년 공연 </b>
👁 조회수: 1572  |  🎟 예매코드: <code>25004150</code>  |  📌마지막 티켓오픈
───────────────────
<b>🟠 내일 [14:00]</b>
<b>뮤지컬 〈플레임즈〉 </b>
👁 조회수: 1372  |  🎟 예매코드: <code>25007077</code>  |  📌2차티켓오픈
───────────────────

<b>🎫 2025년 06월 15일 티켓 오픈 정보 🎫</b>

<b>🟠 내일 [11:00]</b>
<b>뮤지컬 〈사랑의 하츄핑〉 - 서울앵콜 </b>
👁 조회수: 1109  |  🎟 예매코

In [16]:
import os
import requests
import json
import time
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv

url = "https://tickets.interpark.com/contents/api/open-notice/notice-list"

params = {
    "goodsGenre": "ALL", 
    "goodsRegion": "ALL",
    "offset": 0,
    "pageSize": 50,
    "sorting": "OPEN_ASC"
}

headers = {
    "host": "tickets.interpark.com",
    "sec-ch-ua-platform": "Windows", 
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36",
    "accept": "application/json, text/plain, */*",
    "sec-ch-ua": "\"Chromium\";v=\"134\", \"Not:A-Brand\";v=\"24\", \"Google Chrome\";v=\"134\"",
    "sec-ch-ua-mobile": "?0",
    "sec-fetch-site": "same-origin",
    "sec-fetch-mode": "cors", 
    "sec-fetch-dest": "empty",
    "referer": "https://tickets.interpark.com/contents/notice",
    "accept-encoding": "gzip, deflate, br, zstd",
    "accept-language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7"
}

try:
    response = requests.get(url, params=params, headers=headers)
    data = response.json()
    today_date = datetime.now().strftime('%Y년 %m월 %d일')
    
    # HOT 티켓만 필터링하여 데이터 추출
    hot_tickets = []
    
    for ticket in data:
        if ticket['isHot'] == True:
            title = ticket['title']
            if len(title) > 40:
                title = title[:40] + "..."
                
            ticket_info = {
                '오픈시간': ticket['openDateStr'],
                '조회수': ticket['viewCount'],
                '예매타입': ticket['openTypeStr'],
                '제목': title,
                '예매코드': ticket['goodsCode'],
                '멀티오픈': ticket['hasMultipleOpenDates'],
                '장르': ticket['goodsGenreStr'],
                '지역': ticket['goodsRegionStr'],
                '공연장': ticket['venueName'],
                'Image': ticket['posterImageUrl']
            }
            hot_tickets.append(ticket_info)
    
    # 데이터프레임 생성
    df = pd.DataFrame(hot_tickets)
    
    # 데이터프레임 출력
    print("\n=== HOT 티켓 목록 ===")
    print(df)
    
except Exception as e:
    print(f"오류 발생: {e}")


=== HOT 티켓 목록 ===
                   오픈시간    조회수       예매타입  \
0   2025-06-16 15:00:00   7200       일반예매   
1   2025-06-16 15:00:00   3436       일반예매   
2   2025-06-16 17:00:00   3324       일반예매   
3   2025-06-16 19:00:00  35993       일반예매   
4   2025-06-16 19:00:00   9451       일반예매   
5   2025-06-16 20:00:00   5090       일반예매   
6   2025-06-16 20:00:00  19627       일반예매   
7   2025-06-17 14:00:00   7379     1차티켓오픈   
8   2025-06-17 20:00:00   8278       일반예매   
9   2025-06-18 14:00:00   3884       일반예매   
10  2025-06-18 20:00:00  82635       일반예매   
11  2025-06-18 20:00:00  33231       일반예매   
12  2025-06-19 11:00:00   4166  마지막 티켓오픈    

                                             제목      예매코드   멀티오픈   장르  지역  \
0                        2025 여수 K-메가아일랜드 페스티벌             False  콘서트  전남   
1            2025 여수 K-메가아일랜드 페스티벌 현역가왕2 in 여수             False  콘서트  전남   
2                 소란 여름 클럽투어 ＇BUCKETLIST＇ - 서울             False  콘서트  서울   
3   KISS OF LIFE 1st WORLD TOUR 〈KISS ROAD

In [20]:
import os
import tweepy
from dotenv import load_dotenv

# 환경변수 로드
load_dotenv()

def test_twitter_connection():
    """트위터 API 연결 테스트"""
    try:
        print("🔗 트위터 API 연결 테스트 중...")
        
        # API v1.1 (이미지 업로드용)
        auth = tweepy.OAuthHandler(
            os.getenv('TWITTER_API_KEY'),
            os.getenv('TWITTER_API_SECRET')
        )
        auth.set_access_token(
            os.getenv('TWITTER_ACCESS_TOKEN'),
            os.getenv('TWITTER_ACCESS_TOKEN_SECRET')
        )
        api = tweepy.API(auth)
        
        # API v2 (트윗 게시용)
        client = tweepy.Client(
            bearer_token=os.getenv('TWITTER_BEARER_TOKEN'),
            consumer_key=os.getenv('TWITTER_API_KEY'),
            consumer_secret=os.getenv('TWITTER_API_SECRET'),
            access_token=os.getenv('TWITTER_ACCESS_TOKEN'),
            access_token_secret=os.getenv('TWITTER_ACCESS_TOKEN_SECRET'),
            wait_on_rate_limit=True
        )
        
        # 연결 확인 (본인 정보 조회)
        me = client.get_me()
        print(f"✅ 연결 성공! 계정: @{me.data.username}")
        return api, client
        
    except Exception as e:
        print(f"❌ 연결 실패: {e}")
        return None, None

def post_simple_tweet(client):
    """간단한 텍스트 트윗 테스트"""
    try:
        print("\n📝 텍스트 트윗 테스트 중...")
        
        test_text = """🎫 티켓팅 봇 테스트

대리 티켓팅 서비스 테스트 중입니다.
친절한 상담: https://open.kakao.com/o/sAJ8m2Ah

#티켓팅 #테스트"""
        
        response = client.create_tweet(text=test_text)
        print(f"✅ 텍스트 트윗 성공! ID: {response.data['id']}")
        return True
        
    except Exception as e:
        print(f"❌ 텍스트 트윗 실패: {e}")
        return False

def post_tweet_with_image(api, client, image_path):
    """이미지가 포함된 트윗 테스트"""
    try:
        print(f"\n🖼️ 이미지 트윗 테스트 중... ({image_path})")
        
        # 이미지 파일 존재 확인
        if not os.path.exists(image_path):
            print(f"❌ 이미지 파일이 없습니다: {image_path}")
            return False
        
        # 이미지 업로드
        print("📤 이미지 업로드 중...")
        media = api.media_upload(image_path)
        print(f"✅ 이미지 업로드 성공! Media ID: {media.media_id}")
        
        # 트윗 텍스트
        test_text = """🎫 이미지 첨부 테스트

포스터와 함께 티켓팅 정보를 공유합니다.
대리 티켓팅 문의: https://open.kakao.com/o/sAJ8m2Ah

#티켓팅 #이미지테스트"""
        
        # 이미지와 함께 트윗 게시
        response = client.create_tweet(text=test_text, media_ids=[media.media_id])
        print(f"✅ 이미지 트윗 성공! ID: {response.data['id']}")
        return True
        
    except Exception as e:
        print(f"❌ 이미지 트윗 실패: {e}")
        return False

def main():
    """메인 테스트 함수"""
    print("🐦 트위터 API 테스트 시작\n")
    
    # 환경변수 확인
    required_vars = [
        'TWITTER_API_KEY',
        'TWITTER_API_SECRET', 
        'TWITTER_ACCESS_TOKEN',
        'TWITTER_ACCESS_TOKEN_SECRET',
        'TWITTER_BEARER_TOKEN'
    ]
    
    missing_vars = [var for var in required_vars if not os.getenv(var)]
    if missing_vars:
        print(f"❌ 환경변수가 설정되지 않았습니다: {', '.join(missing_vars)}")
        print("📋 .env 파일에 트위터 API 키들을 설정해주세요.")
        return
    
    print("✅ 모든 환경변수가 설정되어 있습니다.\n")
    
    # 1. API 연결 테스트
    api, client = test_twitter_connection()
    if not client:
        return
    
    # 2. 텍스트 트윗 테스트
    text_success = post_simple_tweet(client)
    
    # 3. 이미지 트윗 테스트 (선택사항)
    image_path = input("\n🖼️ 테스트할 이미지 파일 경로를 입력하세요 (Enter를 누르면 건너뜀): ").strip()
    
    if image_path:
        image_success = post_tweet_with_image(api, client, image_path)
    else:
        print("이미지 테스트를 건너뜁니다.")
        image_success = True
    
    # 결과 요약
    print(f"\n{'='*50}")
    print("📊 테스트 결과 요약:")
    print(f"🔗 API 연결: {'✅ 성공' if client else '❌ 실패'}")
    print(f"📝 텍스트 트윗: {'✅ 성공' if text_success else '❌ 실패'}")
    if image_path:
        print(f"🖼️ 이미지 트윗: {'✅ 성공' if image_success else '❌ 실패'}")
    
    if client and text_success:
        print("\n🎉 트위터 API가 정상적으로 작동합니다!")
        print("이제 실제 크롤러에서 자동 게시 기능을 사용할 수 있습니다.")
    else:
        print("\n⚠️ 문제가 발생했습니다. API 키를 확인해주세요.")

if __name__ == "__main__":
    main()

🐦 트위터 API 테스트 시작

✅ 모든 환경변수가 설정되어 있습니다.

🔗 트위터 API 연결 테스트 중...
❌ 연결 실패: 401 Unauthorized
Unauthorized


In [ ]:
import os
import tweepy
from dotenv import load_dotenv

# 환경변수 로드
load_dotenv()

def debug_environment_variables():
    """환경변수 값 확인 (마스킹하여 출력)"""
    print("🔍 환경변수 확인:")
    
    vars_to_check = [
        'TWITTER_API_KEY',
        'TWITTER_API_SECRET', 
        'TWITTER_ACCESS_TOKEN',
        'TWITTER_ACCESS_TOKEN_SECRET',
        'TWITTER_BEARER_TOKEN'
    ]
    
    for var in vars_to_check:
        value = os.getenv(var)
        if value:
            # 처음 4자리와 마지막 4자리만 보여주기
            if len(value) > 8:
                masked = value[:4] + "*" * (len(value) - 8) + value[-4:]
            else:
                masked = "*" * len(value)
            print(f"✅ {var}: {masked}")
        else:
            print(f"❌ {var}: 설정되지 않음")
    print()

def test_api_v1_only():
    """API v1.1만 테스트"""
    try:
        print("🔗 API v1.1 테스트 중...")
        
        auth = tweepy.OAuthHandler(
            os.getenv('TWITTER_API_KEY'),
            os.getenv('TWITTER_API_SECRET')
        )
        auth.set_access_token(
            os.getenv('TWITTER_ACCESS_TOKEN'),
            os.getenv('TWITTER_ACCESS_TOKEN_SECRET')
        )
        
        api = tweepy.API(auth)
        
        # 간단한 인증 테스트
        user = api.verify_credentials()
        print(f"✅ API v1.1 인증 성공! 사용자: @{user.screen_name}")
        return True
        
    except tweepy.Unauthorized:
        print("❌ API v1.1 인증 실패: 토큰이 유효하지 않습니다.")
        print("💡 해결방법:")
        print("   1. 트위터 개발자 포털에서 Access Token 재발급")
        print("   2. .env 파일의 토큰 값 확인")
        print("   3. 앱 권한이 'Read and Write'로 설정되어 있는지 확인")
        return False
    except Exception as e:
        print(f"❌ API v1.1 오류: {e}")
        return False

def test_api_v2_only():
    """API v2만 테스트"""
    try:
        print("\n🔗 API v2 테스트 중...")
        
        client = tweepy.Client(
            consumer_key=os.getenv('TWITTER_API_KEY'),
            consumer_secret=os.getenv('TWITTER_API_SECRET'),
            access_token=os.getenv('TWITTER_ACCESS_TOKEN'),
            access_token_secret=os.getenv('TWITTER_ACCESS_TOKEN_SECRET')
        )
        
        # 본인 정보 조회
        me = client.get_me()
        print(f"✅ API v2 인증 성공! 사용자: @{me.data.username}")
        return True
        
    except tweepy.Unauthorized:
        print("❌ API v2 인증 실패: 토큰이 유효하지 않습니다.")
        return False
    except Exception as e:
        print(f"❌ API v2 오류: {e}")
        return False

def test_bearer_token():
    """Bearer Token 테스트"""
    try:
        print("\n🔗 Bearer Token 테스트 중...")
        
        client = tweepy.Client(bearer_token=os.getenv('TWITTER_BEARER_TOKEN'))
        
        # 공개 정보 조회 (읽기 전용)
        user = client.get_user(username='gamsahanticket')
        print(f"✅ Bearer Token 인증 성공! 조회된 사용자: @{user.data.username}")
        return True
        
    except Exception as e:
        print(f"❌ Bearer Token 오류: {e}")
        return False

def manual_token_input():
    """수동으로 토큰 입력받아 테스트"""
    print("\n🔧 수동 토큰 입력 모드")
    print("트위터 개발자 포털에서 다음 값들을 복사해서 붙여넣어 주세요:")
    
    api_key = input("API Key: ").strip()
    api_secret = input("API Secret: ").strip()
    access_token = input("Access Token: ").strip()
    access_token_secret = input("Access Token Secret: ").strip()
    
    try:
        auth = tweepy.OAuthHandler(api_key, api_secret)
        auth.set_access_token(access_token, access_token_secret)
        api = tweepy.API(auth)
        
        user = api.verify_credentials()
        print(f"\n✅ 수동 입력 토큰으로 인증 성공! @{user.screen_name}")
        
        # 성공한 값들을 .env 파일 형식으로 출력
        print("\n📋 다음 값들을 .env 파일에 저장하세요:")
        print(f"TWITTER_API_KEY={api_key}")
        print(f"TWITTER_API_SECRET={api_secret}")
        print(f"TWITTER_ACCESS_TOKEN={access_token}")
        print(f"TWITTER_ACCESS_TOKEN_SECRET={access_token_secret}")
        
        return True
        
    except Exception as e:
        print(f"❌ 수동 입력 토큰도 실패: {e}")
        return False

def main():
    """메인 디버깅 함수"""
    print("🐛 트위터 인증 디버깅 시작\n")
    
    # 1. 환경변수 확인
    debug_environment_variables()
    
    # 2. API v1.1 테스트
    v1_success = test_api_v1_only()
    
    # 3. API v2 테스트
    v2_success = test_api_v2_only()
    
    # 4. Bearer Token 테스트
    bearer_success = test_bearer_token()
    
    # 5. 결과 분석
    print(f"\n{'='*50}")
    print("📊 디버깅 결과:")
    print(f"🔑 API v1.1: {'✅ 성공' if v1_success else '❌ 실패'}")
    print(f"🔑 API v2: {'✅ 성공' if v2_success else '❌ 실패'}")
    print(f"🔑 Bearer Token: {'✅ 성공' if bearer_success else '❌ 실패'}")
    
    if not any([v1_success, v2_success]):
        print("\n⚠️ 모든 인증이 실패했습니다.")
        print("💡 가능한 원인:")
        print("   1. 잘못된 토큰 값")
        print("   2. 앱 권한 설정 문제")
        print("   3. 토큰 재발급 후 시간 지연")
        
        manual_input = input("\n🔧 수동으로 토큰을 입력해서 테스트해보시겠습니까? (y/n): ").strip().lower()
        if manual_input == 'y':
            manual_token_input()
    else:
        print("\n🎉 일부 인증이 성공했습니다!")
        if v1_success and v2_success:
            print("트윗 게시 기능을 사용할 수 있습니다.")

if __name__ == "__main__":
    main()

🐛 트위터 인증 디버깅 시작

🔍 환경변수 확인:
✅ TWITTER_API_KEY: cmGo*****************nKln
✅ TWITTER_API_SECRET: oVDA******************************************Tehg
✅ TWITTER_ACCESS_TOKEN: 1932******************************************YezY
✅ TWITTER_ACCESS_TOKEN_SECRET: lxfo*************************************vukl
✅ TWITTER_BEARER_TOKEN: AAAA************************************************************************************************************HLZJ

🔗 API v1.1 테스트 중...
❌ API v1.1 인증 실패: 토큰이 유효하지 않습니다.
💡 해결방법:
   1. 트위터 개발자 포털에서 Access Token 재발급
   2. .env 파일의 토큰 값 확인
   3. 앱 권한이 'Read and Write'로 설정되어 있는지 확인

🔗 API v2 테스트 중...
❌ API v2 인증 실패: 토큰이 유효하지 않습니다.

🔗 Bearer Token 테스트 중...
❌ Bearer Token 오류: 401 Unauthorized
Unauthorized

📊 디버깅 결과:
🔑 API v1.1: ❌ 실패
🔑 API v2: ❌ 실패
🔑 Bearer Token: ❌ 실패

⚠️ 모든 인증이 실패했습니다.
💡 가능한 원인:
   1. 잘못된 토큰 값
   2. 앱 권한 설정 문제
   3. 토큰 재발급 후 시간 지연


In [4]:
import os
import tweepy
from dotenv import load_dotenv
from datetime import datetime
import random

# 환경변수 로드
load_dotenv()

def create_unique_tweet_text():
    """매번 다른 내용의 테스트 트윗 생성"""
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    random_num = random.randint(1000, 9999)
    
    return f"""🤖 X API v2 테스트 #{random_num}

⏰ 테스트 시간: {timestamp}
🔧 OAuth 1.0a User Context 인증 성공!

#TwitterAPI #테스트 #Bot"""

def test_oauth1_with_unique_content():
    """고유한 콘텐츠로 OAuth 1.0a 테스트"""
    try:
        print("🔑 OAuth 1.0a User Context 테스트 중...")
        
        # OAuth 1.0a 설정
        client = tweepy.Client(
            consumer_key=os.getenv('TWITTER_API_KEY'),
            consumer_secret=os.getenv('TWITTER_API_SECRET'),
            access_token=os.getenv('TWITTER_ACCESS_TOKEN'),
            access_token_secret=os.getenv('TWITTER_ACCESS_TOKEN_SECRET'),
            wait_on_rate_limit=True
        )
        
        # 본인 정보 조회로 인증 테스트
        me = client.get_me()
        print(f"✅ OAuth 1.0a 인증 성공! 계정: @{me.data.username}")
        
        # 고유한 콘텐츠로 트윗 생성 테스트
        print("🔍 트윗 생성 권한 확인 중...")
        
        
        # 매번 다른 내용의 트윗 생성
        test_tweet = create_unique_tweet_text()
        print(f"📝 생성할 트윗 내용:\n{test_tweet}\n")
        
        # 실제 트윗 게시
        response = client.create_tweet(text=test_tweet)
        
        print(f"✅ 트윗 게시 성공! Tweet ID: {response.data['id']}")
        print(f"📱 트윗 URL: https://twitter.com/{me.data.username}/status/{response.data['id']}")
        
        return True, client
        
    except tweepy.Forbidden as e:
        error_msg = str(e)
        if "duplicate content" in error_msg.lower():
            print(f"⚠️ 중복 콘텐츠 오류: {e}")
            print("💡 이는 정상적인 보안 기능입니다.")
            print("✅ 권한은 정상적으로 작동하고 있습니다!")
            return True, None  # 권한은 정상이므로 True 반환
        else:
            print(f"❌ 권한 오류: {e}")
            print("💡 해결방법:")
            print("   1. 앱 권한을 'Read and write'로 설정")
            print("   2. Access Token 재발급")
            return False, None
        
    except tweepy.Unauthorized as e:
        print(f"❌ 인증 오류: {e}")
        print("💡 해결방법:")
        print("   1. API 키와 토큰 값 재확인")
        print("   2. 토큰이 올바른 앱에서 발급되었는지 확인")
        return False, None
        
    except Exception as e:
        print(f"❌ 기타 오류: {e}")
        return False, None

def test_image_upload_with_unique_content():
    """고유한 콘텐츠로 이미지 업로드 및 트윗 테스트"""
    try:
        print("\n🖼️ 이미지 업로드 및 트윗 테스트...")
        
        # OAuth 1.0a 클라이언트
        client = tweepy.Client(
            consumer_key=os.getenv('TWITTER_API_KEY'),
            consumer_secret=os.getenv('TWITTER_API_SECRET'),
            access_token=os.getenv('TWITTER_ACCESS_TOKEN'),
            access_token_secret=os.getenv('TWITTER_ACCESS_TOKEN_SECRET')
        )
        
        # OAuth 1.0a API (이미지 업로드용)
        auth = tweepy.OAuthHandler(
            os.getenv('TWITTER_API_KEY'),
            os.getenv('TWITTER_API_SECRET')
        )
        auth.set_access_token(
            os.getenv('TWITTER_ACCESS_TOKEN'),
            os.getenv('TWITTER_ACCESS_TOKEN_SECRET')
        )
        api = tweepy.API(auth)
        
        # 테스트 이미지 경로 입력받기
        image_path = input("테스트할 이미지 파일 경로를 입력하세요 (Enter를 누르면 건너뜀): ").strip()
        
        if not image_path:
            print("이미지 테스트를 건너뜁니다.")
            return True
            
        if not os.path.exists(image_path):
            print(f"❌ 이미지 파일이 없습니다: {image_path}")
            return False
        
        # 이미지 업로드 (API v1.1 사용)
        print("📤 이미지 업로드 중...")
        media = api.media_upload(image_path)
        print(f"✅ 이미지 업로드 성공! Media ID: {media.media_id}")
        
        # 고유한 콘텐츠로 이미지 트윗 생성
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        random_num = random.randint(1000, 9999)
        
        tweet_text = f"""🎫 이미지 첨부 테스트 #{random_num}

⏰ 테스트 시간: {timestamp}
🖼️ OAuth 1.0a로 이미지 업로드 및 트윗 게시 성공!

#TwitterAPI #OAuth1 #이미지테스트"""
        
        response = client.create_tweet(text=tweet_text, media_ids=[media.media_id])
        print(f"✅ 이미지 트윗 게시 성공! Tweet ID: {response.data['id']}")
        
        return True
        
    except tweepy.Forbidden as e:
        if "duplicate content" in str(e).lower():
            print(f"⚠️ 중복 콘텐츠 오류: {e}")
            print("✅ 이미지 업로드 권한은 정상입니다!")
            return True
        else:
            print(f"❌ 이미지 트윗 실패: {e}")
            return False
        
    except Exception as e:
        print(f"❌ 이미지 트윗 실패: {e}")
        return False

def create_realistic_ticket_tweet():
    """실제 티켓팅 트윗과 유사한 형태 생성"""
    timestamp = datetime.now().strftime("%m월 %d일 %H:%M")
    
    artists = ["세븐틴", "BTS", "NewJeans", "IVE", "aespa"]
    venues = ["올림픽공원 체조경기장", "KSPO DOME", "잠실실내체조경기장", "블루스퀘어"]
    
    artist = random.choice(artists)
    venue = random.choice(venues)
    
    return f"""{artist} 콘서트 {venue}

대리 티켓팅 진행
최근 세븐틴 / BTS / 블랙핑크 댈티 성공경력

선착순 할인 이벤트:
VIP 잡아도 수고비 5만원 선입금, 실패시 수고비 전액환불

🕐 테스트: {timestamp}

친절한 상담: https://open.kakao.com/o/sAJ8m2Ah

#{artist} #콘서트 #티켓팅 #대리티켓팅"""

def test_realistic_tweet():
    """실제 사용할 형태의 트윗 테스트"""
    try:
        print("\n🎫 실제 티켓팅 트윗 형태 테스트...")
        
        client = tweepy.Client(
            consumer_key=os.getenv('TWITTER_API_KEY'),
            consumer_secret=os.getenv('TWITTER_API_SECRET'),
            access_token=os.getenv('TWITTER_ACCESS_TOKEN'),
            access_token_secret=os.getenv('TWITTER_ACCESS_TOKEN_SECRET'),
            wait_on_rate_limit=True
        )
        
        # 실제 사용할 트윗 형태 생성
        realistic_tweet = create_realistic_ticket_tweet()
        print(f"📝 생성할 실제 형태 트윗:\n{realistic_tweet}\n")
        
        confirm = input("🤔 이 트윗을 실제로 게시하시겠습니까? (y/n): ").strip().lower()
        if confirm != 'y':
            print("트윗 게시를 취소했습니다.")
            return True
        
        # 트윗 게시
        response = client.create_tweet(text=realistic_tweet)
        print(f"✅ 실제 형태 트윗 게시 성공! Tweet ID: {response.data['id']}")
        
        return True
        
    except Exception as e:
        print(f"❌ 실제 형태 트윗 실패: {e}")
        return False

def main():
    """메인 테스트 함수"""
    print("🐦 중복 콘텐츠 방지 X API 테스트 시작\n")
    
    print("💡 참고: 트위터는 같은 내용의 트윗을 연속으로 게시하는 것을 금지합니다.")
    print("이는 스팸 방지를 위한 정상적인 보안 기능입니다.\n")
    
    # 환경변수 확인
    required_vars = [
        'TWITTER_API_KEY',
        'TWITTER_API_SECRET', 
        'TWITTER_ACCESS_TOKEN',
        'TWITTER_ACCESS_TOKEN_SECRET'
    ]
    
    missing_vars = [var for var in required_vars if not os.getenv(var)]
    if missing_vars:
        print(f"❌ 환경변수가 설정되지 않았습니다: {', '.join(missing_vars)}")
        return
    
    print("✅ 필수 환경변수가 설정되어 있습니다.")
    
    # 고유한 콘텐츠로 OAuth 1.0a 인증 테스트
    success, client = test_oauth1_with_unique_content()
    
    if success:
        print(f"\n🎉 OAuth 1.0a User Context 권한이 정상적으로 작동합니다!")
        print("X API v2를 사용하여 트윗을 게시할 수 있습니다.")
        
        # 추가 테스트 선택
        print(f"\n📋 추가 테스트 옵션:")
        print("1. 이미지 업로드 테스트")
        print("2. 실제 티켓팅 트윗 형태 테스트")
        print("3. 테스트 종료")
        
        choice = input("선택하세요 (1/2/3): ").strip()
        
        if choice == '1':
            test_image_upload_with_unique_content()
        elif choice == '2':
            test_realistic_tweet()
        else:
            print("테스트를 종료합니다.")
            
    else:
        print(f"\n❌ 권한 문제가 있습니다. 설정을 다시 확인해주세요.")

if __name__ == "__main__":
    main()

🐦 중복 콘텐츠 방지 X API 테스트 시작

💡 참고: 트위터는 같은 내용의 트윗을 연속으로 게시하는 것을 금지합니다.
이는 스팸 방지를 위한 정상적인 보안 기능입니다.

✅ 필수 환경변수가 설정되어 있습니다.
🔑 OAuth 1.0a User Context 테스트 중...
✅ OAuth 1.0a 인증 성공! 계정: @gamsahanticket
🔍 트윗 생성 권한 확인 중...
📝 생성할 트윗 내용:
🤖 X API v2 테스트 #2851

⏰ 테스트 시간: 2025-06-15 19:57:29
🔧 OAuth 1.0a User Context 인증 성공!

#TwitterAPI #테스트 #Bot

✅ 트윗 게시 성공! Tweet ID: 1934203602513625574
📱 트윗 URL: https://twitter.com/gamsahanticket/status/1934203602513625574

🎉 OAuth 1.0a User Context 권한이 정상적으로 작동합니다!
X API v2를 사용하여 트윗을 게시할 수 있습니다.

📋 추가 테스트 옵션:
1. 이미지 업로드 테스트
2. 실제 티켓팅 트윗 형태 테스트
3. 테스트 종료

🖼️ 이미지 업로드 및 트윗 테스트...
📤 이미지 업로드 중...
✅ 이미지 업로드 성공! Media ID: 1934203706553323520
✅ 이미지 트윗 게시 성공! Tweet ID: 1934203708654715308
